# LangChain

In [1]:
! pip list | grep langchain

langchain                     0.0.274


In [2]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo")

In [8]:
response = chat([HumanMessage(content="Translate this sentence from English to French: I love programming.")])
response.content

"J'adore programmer."

In [11]:
response.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'messages', 'AIMessage'],
 'kwargs': {'content': "J'adore programmer.", 'additional_kwargs': {}}}

# Memory

Memory allows the AI to remember the context of human interactions. This memory is preserved by creating summaries.

In [15]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("Hi! Nice to meet you, I'm Shem and I am a programer!")
memory.chat_memory.add_ai_message("Oh! Nice to meet you too, I'm Audrey and I also a programer!")

In [16]:
memory.load_memory_variables({}) # when you use Buffer Memory, memory will completly saved.

{'history': "Human: Hi! Nice to meet you, I'm Shem and I am a programer!\nAI: Oh! Nice to meet you too, I'm Audrey and I also a programer!"}

In [20]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.chat_memory.add_user_message("Hi! Nice to meet you, I'm Shem and I am a programer!")
memory.chat_memory.add_ai_message("Oh! Nice to meet you too, I'm Audrey and I also a programer!")
memory.chat_memory.add_user_message("Hi! Nice to meet you, I'm Jack and I am a house keeper!")
memory.chat_memory.add_ai_message("Oh! Nice to meet you too, I'm Black and I also a house keeper!")
memory.load_memory_variables({}) # When you use a window buffer, the memory is saved based on the 'k' steps you've set.

{'history': "Human: Hi! Nice to meet you, I'm Jack and I am a house keeper!\nAI: Oh! Nice to meet you too, I'm Black and I also a house keeper!"}

In [32]:
from langchain.llms import OpenAI
from langchain.memory import ConversationSummaryMemory

llm = OpenAI(temperature=0)
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)
memory.save_context(inputs={"User":"Hi! Nice to meet you, I'm Shem and I am a programer!"}, outputs={"AI":"Oh! Nice to meet you too, I'm Audrey and I also a programer!"})

In [33]:
memory.load_memory_variables({})

{'chat_history': [SystemMessage(content='\nThe human introduces themselves as Shem, a programmer, and the AI introduces themselves as Audrey, also a programmer.', additional_kwargs={})]}

## Impletment chat with memory

Using the SummaryMemory we just created, we can ask the AI for the name we added to the memory.

In [34]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain

# LLM
llm = ChatOpenAI()

# Prompt 
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "Do you remember whats my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
System: 
The human introduces themselves as Shem, a programmer, and the AI introduces themselves as Audrey, also a programmer.
Human: Do you remember whats my name?

> Finished chain.


{'question': 'Do you remember whats my name?',
 'chat_history': [SystemMessage(content='\nThe human introduces themselves as Shem, a programmer, and the AI introduces themselves as Audrey, also a programmer.', additional_kwargs={})],
 'text': "Yes, your name is Shem. It's nice to meet you, Shem! How can I assist you today?"}

# Retrieval-Arguments Generative, RAG

## Dataset

In [18]:
! curl -L -o ../data/TSMC2023Q1.zip "https://drive.google.com/uc?export=download&id=1Z_ww5ZASdIq0uZrg8jt_d7E_PzFQdYH-"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:05 --:--:--     0 0    0     0      0      0 --:--:--  0:00:04 --:--:--     0
100 10.1M  100 10.1M    0     0  1474k      0  0:00:07  0:00:07 --:--:-- 10.8M


In [23]:
! cd ../data && unzip TSMC2023Q1.zip

Archive:  TSMC2023Q1.zip
  inflating: TSMC2023Q1/20230330 TSMC Citi.pdf  
  inflating: TSMC2023Q1/20230331 TSMC UBS.pdf  
  inflating: TSMC2023Q1/20230406 TSMC MS.pdf  
  inflating: TSMC2023Q1/20230410 TSMC DW.pdf  
  inflating: TSMC2023Q1/20230410 TSMC HTI.pdf  
  inflating: TSMC2023Q1/20230410 TSMC JPM.pdf  
  inflating: TSMC2023Q1/20230411 TSMC GS.pdf  
  inflating: TSMC2023Q1/20230412 TSMC CL.pdf  
  inflating: TSMC2023Q1/20230414 TSMC NMR.pdf  
  inflating: TSMC2023Q1/20230417 TSMC HSBC.pdf  


In [3]:
! pip list | grep pypdf

pypdf                         3.15.4


In [3]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader('../data/TSMC2023Q1/20230330 TSMC Citi.pdf')

In [4]:
pages = loader.load()
len(pages) # pages of the pdf

15

In [5]:
print(pages[0].page_content) # content of page 1

See Appendix A-1 for Analyst Certification, Important Disclosures and Research Analyst Affiliations.
Citi Research is a division of Citigroup Global Markets Inc. (the "Firm"), which does and seeks to do business with companies covered in its research 
reports. As a result, investors should be aware that the Firm may have a conflict of interest that could affect the objectivity of this report. Investors should 
consider this report as only a single factor in making their investment decision. Certain products (not inconsistent with the author's published research) are 
available only on Citi's portals.30 Mar 2023 07:47:50 ET │ 15 pages    TSMC (2330.TW)Assessing the Potential Upside from AI CITI'S TAKE AI is witnessing a potential “iPhone moment” and should be a L-T catalyst in 
the semi industry, despite still small contribution at current early stage. nVidia 
dominates the data center GPU market for now, while we expect to see more 
AI chips in future designed by hyperscalers, includin

## Splitting posts into chunk

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/shemyu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
from langchain.text_splitter import NLTKTextSplitter
splitter = NLTKTextSplitter(chunk_size = 1000, chunk_overlap = 200) # 1000 per chunk and overlap with 20%
chunks = splitter.split_text(pages[0].page_content) # split page 1

In [8]:
len(chunks) # page 1 is split into 5 chunks

5

## Vector Database

In [9]:
! pip list | grep chromadb

chromadb                      0.4.7


In [10]:
from langchain.vectorstores import Chroma # Langchian imtegrated Chroma
from langchain.embeddings.openai import OpenAIEmbeddings # Encoder

embedding = OpenAIEmbeddings()

persist_directory = './test_chroma'
vectordb = Chroma.from_texts(
    texts=chunks, # the Chunks input
    embedding = embedding, # Embedding Engine
    persist_directory=persist_directory # Storage placed
)

## Retrieval by AI

In [15]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)
result = qa_chain({"query": '#zh-tw 請問citi 對台積電的看法? 請用正體中文'})

In [16]:
result

{'query': '#zh-tw 請問citi 對台積電的看法? 請用正體中文',
 'result': 'Citi對台積電的看法是積極的。他們預期台積電在人工智慧和高性能運算方面的項目將帶來更好的2024年展望。他們重申買入評級，目標價維持在NT$620。他們認為台積電在人工智慧領域有著巨大的潛力，儘管目前貢獻仍然較小。'}

In [93]:
from langchain.prompts import PromptTemplate


# LLM
llm = ChatOpenAI(model_name="gpt-4", temperature=0, verbose=True)

# Prompt 
message_template = """你是一名專業投資理財顧問，
請依據財報內容給予專業的理財建議。
給出的財報內容分為三大段落：
1. 你的文字建議，文字字數請控制在40字內
2. 第二部分重點財報指標，請整理成視覺化的表格

現在請用正體中文回應我的問題：{question}
"""

prompt = PromptTemplate(
    template=message_template,
    input_variables=["question"],
)

In [94]:
# chain_type_kwargs = {"prompt": prompt}
retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
    # chain_type_kwargs=chain_type_kwargs,
)

In [95]:
query_str = message_template.format(question="請問citi 對台積電的看法? 請用正體中文")
result = retriever({"query": query_str})

In [96]:
print(result["result"]) 

1. 建議：Citi對台積電的看法樂觀，預期其2024年表現將因AI和HPC的穩定產品線而改善，並重申買入評級，目標價格為NT$620。

2. 財報指標視覺化表格：

| 年份 | 淨利潤 (NT$M) | 每股盈餘 (NT$) | EPS成長 (%) | P/E | P/B | ROE (%) | 殖利率 (%) |
|------|--------------|--------------|-------------|-----|-----|---------|------------|
| 2021A | 596,540 | 23.00 | 15.2 | 23.3 | 6.4 | 29.7 | 2.1 |
| 2022A | 1,016,530 | 39.20 | 70.4 | 13.6 | 4.7 | 39.6 | 2.1 |
| 2023E | 872,096 | 33.63 | -14.2 | 15.9 | 3.9 | 26.9 | 2.2 |
| 2024E | 1,019,383 | 39.31 | 16.9 | 13.6 | 3.3 | 26.3 | 2.3 |
| 2025E | 1,276,397 | 49.22 | 25.2 | 10.9 | 2.7 | 27.2 | 2.4 |


### GPT-4 表現真的十分穩定優異

| 年份 | 淨利潤 (NT$M) | 每股盈餘 (NT$) | EPS成長 (%) | P/E | P/B | ROE (%) | 殖利率 (%) |
|------|--------------|--------------|-------------|-----|-----|---------|------------|
| 2021A | 596,540 | 23.00 | 15.2 | 23.3 | 6.4 | 29.7 | 2.1 |
| 2022A | 1,016,530 | 39.20 | 70.4 | 13.6 | 4.7 | 39.6 | 2.1 |
| 2023E | 872,096 | 33.63 | -14.2 | 15.9 | 3.9 | 26.9 | 2.2 |
| 2024E | 1,019,383 | 39.31 | 16.9 | 13.6 | 3.3 | 26.3 | 2.3 |
| 2025E | 1,276,397 | 49.22 | 25.2 | 10.9 | 2.7 | 27.2 | 2.4 |

In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate


# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, n=5, verbose=True)

# Prompt 
message_template = """你是一名專業投資理財顧問，
請依據財報內容給予專業的理財建議。
給出的財報內容分為三大段落：
1. 你的文字建議，文字字數請控制在40字內
2. 第二部分重點財報指標，請整理成視覺化的表格

現在請用正體中文回應我的問題：{question}
"""

prompt = PromptTemplate(
    template=message_template,
    input_variables=["question"],
)

In [25]:
from langchain.chains import RetrievalQA


retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
)

In [26]:
query_str = message_template.format(question="請問citi 對台積電的看法? 請用正體中文")
result = retriever({"query": query_str})

In [27]:
result # n 參數無效

{'query': '你是一名專業投資理財顧問，\n請依據財報內容給予專業的理財建議。\n給出的財報內容分為三大段落：\n1. 你的文字建議，文字字數請控制在40字內\n2. 第二部分重點財報指標，請整理成視覺化的表格\n\n現在請用正體中文回應我的問題：請問citi 對台積電的看法? 請用正體中文\n',
 'result': '1. 建議：根據財報內容，建議投資者買進台積電股票，預期未來表現良好。\n\n2. 財報指標：\n   - 2021年淨利潤：NT$596,540M\n   - 2021年每股稀釋盈利：NT$23.00\n   - 2021年盈利增長率：15.2%\n   - 2021年市盈率：23.3倍\n   - 2021年市價淨值比：6.4倍\n   - 2021年股東權益報酬率：29.7%\n   - 2021年股息收益率：2.1%'}

In [28]:
print(result["result"])

1. 建議：根據財報內容，建議投資者買進台積電股票，預期未來表現良好。

2. 財報指標：
   - 2021年淨利潤：NT$596,540M
   - 2021年每股稀釋盈利：NT$23.00
   - 2021年盈利增長率：15.2%
   - 2021年市盈率：23.3倍
   - 2021年市價淨值比：6.4倍
   - 2021年股東權益報酬率：29.7%
   - 2021年股息收益率：2.1%


### GPT-3.5-turbo 表現較不穩定
計價便宜但是結果也相對沒這麼穩定，可能要用一些 Try Catch 解決

1. 建議：根據財報內容，建議投資者買進台積電股票，預期未來表現良好。

2. 財報指標：
   - 2021年淨利潤：NT$596,540M
   - 2021年每股稀釋盈利：NT$23.00
   - 2021年盈利增長率：15.2%
   - 2021年市盈率：23.3倍
   - 2021年市價淨值比：6.4倍
   - 2021年股東權益報酬率：29.7%
   - 2021年股息收益率：2.1%

In [29]:
query_str = message_template.format(question="請問citi 對台積電的看法?")
result = retriever({"query": query_str})

In [31]:
print(result["result"])

1. 建議：根據財報內容，Citi對台積電持有正向看法，預期2024年將有更好的前景，並推薦買進，目標價維持在NT$620。

2. 重點財報指標：

| 年份 | 淨利潤 (NT$M) | 每股盈餘 (NT$) | 盈餘成長率 (%) | 市盈率 (x) | 淨資產價值比率 (x) | 股東權益報酬率 (%) | 預期股息率 (%) |
|------|--------------|----------------|----------------|------------|------------------|--------------------|----------------|
| 2021A | 596,540 | 23.00 | 15.2 | 23.3 | 6.4 | 29.7 | 2.1 |
| 2022A | 1,016,530 | 39.20 | 70.4 | 13.6 | 4.7 | 39.6 | 2.1 |
| 2023E | 872,096 | 33.63 | -14.2 | 15.9 | 3.9 | 26.9 | 2.2 |
| 2024E | 1,019,383 | 39.31 | 16.9 | 13.6 | 3.3 | 26.3 | 2.3 |
| 2025E | 1,276,397 | 49.22 | 25.2 | 10.9 | 2.7 | 27.2 | 2.4 |

以上資料來源：Citi Research


### GPT-3.5-turbo 重複嘗試後成功輸出表格

#### Result of GPT-3.5-turbo
| 年份 | 淨利潤 (NT$M) | 每股盈餘 (NT$) | 盈餘成長率 (%) | 市盈率 (x) | 淨資產價值比率 (x) | 股東權益報酬率 (%) | 預期股息率 (%) |
|------|--------------|----------------|----------------|------------|------------------|--------------------|----------------|
| 2021A | 596,540 | 23.00 | 15.2 | 23.3 | 6.4 | 29.7 | 2.1 |
| 2022A | 1,016,530 | 39.20 | 70.4 | 13.6 | 4.7 | 39.6 | 2.1 |
| 2023E | 872,096 | 33.63 | -14.2 | 15.9 | 3.9 | 26.9 | 2.2 |
| 2024E | 1,019,383 | 39.31 | 16.9 | 13.6 | 3.3 | 26.3 | 2.3 |
| 2025E | 1,276,397 | 49.22 | 25.2 | 10.9 | 2.7 | 27.2 | 2.4 |


#### Result of GPT-4
| 年份 | 淨利潤 (NT$M) | 每股盈餘 (NT$) | EPS成長 (%) | P/E | P/B | ROE (%) | 殖利率 (%) |
|------|--------------|--------------|-------------|-----|-----|---------|------------|
| 2021A | 596,540 | 23.00 | 15.2 | 23.3 | 6.4 | 29.7 | 2.1 |
| 2022A | 1,016,530 | 39.20 | 70.4 | 13.6 | 4.7 | 39.6 | 2.1 |
| 2023E | 872,096 | 33.63 | -14.2 | 15.9 | 3.9 | 26.9 | 2.2 |
| 2024E | 1,019,383 | 39.31 | 16.9 | 13.6 | 3.3 | 26.3 | 2.3 |
| 2025E | 1,276,397 | 49.22 | 25.2 | 10.9 | 2.7 | 27.2 | 2.4 |

### Try to fix the problem by prompt 

In [41]:
prompt_template_string = """你是一名專業投資理財顧問，
請依據財報內容給予專業的理財建議。
給出的財報內容分為三大段落：
1. 你的文字建議，文字字數請控制在40字內
2. 第二部分重點財報指標，請整理成視覺化的表格

現在請用正體中文回應我的問題：{question}
"""
def qa_chat(question):
    query_str = prompt_template_string.format(question=question)
    result = retriever({"query": query_str})
    return result

### 以原 Prompt 進行實驗

In [44]:
test_times = 10
for _ in range(test_times):
    print(qa_chat("請問citi 對台積電的看法?")["result"])

1. 建議：根據財報內容，Citi對台積電持有正向看法，預期2024年將有更好的前景，並推薦買進，目標價維持在NT$620。

2. 重點財報指標：

| 年份 | 淨利潤 (NT$M) | 每股盈餘 (NT$) | 盈餘成長率 (%) | 市盈率 (x) | 淨資產價值比率 (x) | 股東權益報酬率 (%) | 預期股息率 (%) |
|------|--------------|----------------|----------------|------------|------------------|--------------------|----------------|
| 2021A | 596,540 | 23.00 | 15.2 | 23.3 | 6.4 | 29.7 | 2.1 |
| 2022A | 1,016,530 | 39.20 | 70.4 | 13.6 | 4.7 | 39.6 | 2.1 |
| 2023E | 872,096 | 33.63 | -14.2 | 15.9 | 3.9 | 26.9 | 2.2 |
| 2024E | 1,019,383 | 39.31 | 16.9 | 13.6 | 3.3 | 26.3 | 2.3 |
| 2025E | 1,276,397 | 49.22 | 25.2 | 10.9 | 2.7 | 27.2 | 2.4 |

以上資料來源：Citi Research
1. 建議：根據財報內容，Citi對台積電持有正向看法，預期2024年將有更好的前景，並推薦買進，目標價維持在NT$620。

2. 重點財報指標：

| 年份 | 淨利潤 (NT$M) | 每股盈餘 (NT$) | 盈餘成長率 (%) | 市盈率 (x) | 淨資產價值比率 (x) | 股東權益報酬率 (%) | 預期股息率 (%) |
|------|--------------|----------------|----------------|------------|------------------|--------------------|----------------|
| 2021A | 596,540 | 23.00 | 15.2 | 23.3 | 6.4 | 29.7 | 2.1 |
| 2022A

`大概 10 次當中錯誤了 3 次`

### 以改善後的 Prompt 進行實驗

In [48]:
prompt_template_string = """你是一名專業投資理財顧問，
請依據財報內容給予專業的理財建議。你除了以下兩點外，不會回應任何其他的內容，也只會照著以下規範的 template 進行輸出。
給出的財報內容分為三大段落：
1. 你的文字建議，文字字數請控制在40字內
2. 第二部分重點財報指標，請整理成視覺化的表格，表格的 template 如下：
| 年份 | 淨利潤 (NT$M) | 每股盈餘 (NT$) | EPS成長 (%) | P/E | P/B | ROE (%) | 殖利率 (%) |
|-|-|-|-|-|-|-|-|
| 20xxA |  |  |  |  |  |  |  |

現在請用正體中文回應我的問題：{question}"""
def qa_chat(question):
    query_str = prompt_template_string.format(question=question)
    result = retriever({"query": query_str})
    return result

In [49]:
test_times = 10
for _ in range(test_times):
    print(qa_chat("請問citi 對台積電的看法?")["result"])

1. 建議：根據財報內容，Citi對台積電持有正向看法，預期2024年將有更好的前景，並且該公司在人工智能和高性能運算方面有穩固的產品線。建議繼續買入，目標價格維持在NT$620。

2. 財報指標表格：
| 年份 | 淨利潤 (NT$M) | 每股盈餘 (NT$) | EPS成長 (%) | P/E | P/B | ROE (%) | 殖利率 (%) |
|-|-|-|-|-|-|-|-|
| 2021A | 596,540 | 23.00 | 15.2 | 23.3 | 6.4 | 29.7 | 2.1 |
| 2022A | 1,016,530 | 39.20 | 70.4 | 13.6 | 4.7 | 39.6 | 2.1 |
| 2023E | 872,096 | 33.63 | -14.2 | 15.9 | 3.9 | 26.9 | 2.2 |
| 2024E | 1,019,383 | 39.31 | 16.9 | 13.6 | 3.3 | 26.3 | 2.3 |
| 2025E | 1,276,397 | 49.22 | 25.2 | 10.9 | 2.7 | 27.2 | 2.4 |

請注意，以上建議僅供參考，投資者應綜合考慮其他因素做出投資決策。
1. 建議：根據財報內容，Citi對台積電持有正向看法，預期2024年將有更好的前景，並且該公司在人工智能和高性能運算方面有穩固的產品線。建議繼續買入，目標價格維持在NT$620。

2. 財報指標表格：
| 年份 | 淨利潤 (NT$M) | 每股盈餘 (NT$) | EPS成長 (%) | P/E | P/B | ROE (%) | 殖利率 (%) |
|-|-|-|-|-|-|-|-|
| 2021A | 596,540 | 23.00 | 15.2 | 23.3 | 6.4 | 29.7 | 2.1 |
| 2022A | 1,016,530 | 39.20 | 70.4 | 13.6 | 4.7 | 39.6 | 2.1 |
| 2023E | 872,096 | 33.63 | -14.2 | 15.9 | 3.9 | 26.9 | 2.2 |
| 2024E | 1,019,383 | 39.31 | 16.9 | 13.6 | 3.3 | 26.3 | 2.3 |
| 2025E |

`Prompt 當中給出輸出格式範例時，穩定性明顯上升，表格的呈現 10 次皆為正確。`

### 實驗 Data extraction 並指定可串接應用的 json 格式

In [65]:
prompt_template_string = """請找到 <ques><ques/> 當中的問題，相關的所有與投資有關的資訊。並只會以下方的json格式進行輸出，將資料放在data當中，如果你做不到，請在status key 中回傳 "ERROR" 以方便我在串接時能做錯誤處理。並且你只會輸出 json 格式的 response data，不會給出任合其他字串。
{output_template}
現在請用正體中文回應我的問題：{question}"""
def qa_chat(question, output_template):
    query_str = prompt_template_string.format(question=question, output_template=output_template)
    result = retriever({"query": query_str})
    return result

In [55]:
print(qa_chat("請問citi 對台積電的看法?")["result"], "{'staus':'OK','data':{}}")

{"status":"OK","data":{"Citi對台積電的看法":"Citi認為人工智慧（AI）正在經歷潛在的“iPhone時刻”，並且在半導體行業中應該是一個長期的催化劑，儘管目前在早期階段對半導體行業的貢獻仍然很小。他們預計由於在AI和高性能計算（HPC）領域的穩定項目，2024年的前景將更好。Citi重申對台積電的買入評級，目標價格維持在NT$620。"}}


In [66]:
o_template = "{'staus':'OK','data':{<company_name>:{<財報指標>:<財報指標值>, ...}}}"
print(
    qa_chat(
        "請問citi 對台積電的看法?",
        o_template
    )["result"]
)

{
  "status": "OK",
  "data": {
    "TSMC": {
      "Buy Price (30 Mar 23 13:30)": "NT$535.00",
      "Target price": "NT$620.00",
      "Expected share price return": "15.9%",
      "Expected dividend yield": "2.2%",
      "Expected total return": "18.1%",
      "Market Cap": "NT$13,873,882M US$454,852M",
      "Earnings Summary": {
        "Year to": {
          "Net Profit": "1,016,530",
          "Diluted EPS": "39.20",
          "EPS growth": "70.4%",
          "P/E": "13.6",
          "P/B": "4.7",
          "ROE": "39.6%",
          "Yield": "2.1%"
        }
      }
    }
  }
}


# 結論 20230829

## Positive
### GPT-3.5-turbo 表現穩定 CP 高

- CP 值相較 GPT-4 來得更好
- 跨語言搜尋也穩定正確
  
### 在 data extraction 方面表現優異

- 回傳資料在參數正確的情況下，表現完美: 財報資料 0 失誤
- Prompt tuning 後能正確的 parsing 出 PDF 當中的資料
- 只要穩定回傳 json 規範格式，可以應用於自動化場景中
  
### Promp 品質影響大
- 大部分問題可以不用靠大量資源 tuning model 實現，僅修正 prompt 內容就可以大幅提高正確率
  
## Negative
### 搜尋效率問題
- 結合 GPT api 查詢速度非常慢，一次 request 到回應要約莫 10 秒(狀態優良時，狀態不好會花到 30 秒)
- 直接性的影響 2c 服務串接的品質
### 負面 prompt 影響很小

第二段中間我定義了「除了以下兩點外，不會 ...」的負面指令，然而在第三段落，我又給出需要分為三大段落的指示。

```
你是一名專業投資理財顧問，
請依據財報內容給予專業的理財建議。你除了以下兩點外，不會回應任何其他的內容，也只會照著以下規範的 template 進行輸出。
給出的財報內容分為三大段落：
1. 你的文字建議，文字字數請控制在40字內
2. 第二部分重點財報指標，請整理成視覺化的表格，表格的 template 如下：
| 年份 | 淨利潤 (NT$M) | 每股盈餘 (NT$) | EPS成長 (%) | P/E | P/B | ROE (%) | 殖利率 (%) |
|-|-|-|-|-|-|-|-|
| 20xxA |  |  |  |  |  |  |  |

現在請用正體中文回應我的問題：{question}
```

從 repsponse 結果可以看到 10 次皆為三大段落。實驗過程中也屢屢發現負面 prompt 的效果不顯著。
- 叫他不要做什麼效果不到，盡量要改 prompt 方式為"你只會 ..."